In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import math
import plot_utils
TITLE_FONT = {'fontname': 'Arial', 'fontsize': 12, 'fontweight': 'bold'}
smaller_font = {'fontname': 'Arial', 'fontsize': 10, 'fontweight': 'normal'}
tiny_fontsize = 8
micro_fontsize = 5
plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300

In [ ]:
t_point, T_point, G_point, pH_point = plot_utils.analyse_and_plot("Data/0529-4--pH--G--T.npy", True, 4.1, RTD_constant = 0.3)

In [ ]:
#T related
r = 0.1 # bowl radius
c = 4.2e3 #specific heat capacity of soup
rho = 1e3 # density of soup
l = 0.03 # soup height
h_side = 50 # natural convection coefficient through side
h_top = 50 # natural convection coefficient through top
hob_rate = 7 # hob rate as shown on the hob
# P = 1 # PWM percentage for pot on hob
T_inf = 20 # room temperature

Q_in = 4.88 * 2 ** hob_rate # heat addition rate per unit time


T0 = T_inf

#G related
G_0 = 0.8e-3
alpha_G = 0.8
beta_G = 0.025
gamma_G = 54

#pH related
beta_pH = 0.00585
gamma_pH = 0.9

#n_s related
alpha_t = 10**(-3.5)
k_1 = 0.0001
k_2 = 0.005
k = 1e-6
V_s = 1

def f_G(G_raw, T):
    return G_raw
    c = 0.2
    rate = 0.02    
    return (-c*np.exp(-rate*(T-25))+(c+1)) * G_raw

def simulationStandard(Q, index, salt, vinegar, V_t = 0.4, dt = 0.1):
    alpha_t = 10**(-3.5)
    t = G = pH = n_s = m_s = m_v = 0
    T = T0
    t_total = len(Q) * dt
    counter = 0
    data = []
    salt = salt * 1e-3 # in kg
    vinegar = vinegar * 1e-2 # in kg
    
    
    while t < t_total:
        t += dt
        T += 1/(c*rho*l*math.pi*r**2)*(-(T-T_inf)*(h_top*math.pi*r**2 + 2*h_side*math.pi*r*l) + Q[counter]) * dt
        G_raw = G_0 + alpha_G*m_s + beta_G*m_v + gamma_G*n_s
        G = f_G(G_raw, T)
        n_s += k_1*np.exp(k_2*(T-25))*(alpha_t - n_s*(1/V_t + 1/V_s))
        pH = -np.log10(10e-7 + beta_pH*m_v + gamma_pH*n_s)
        if round(t, 1) in index:
            m_s += float(salt[np.where(index == round(t, 1))])
            m_v += float(vinegar[np.where(index == round(t, 1))])
#             print("m_s, m_v:", m_s, m_v)
        
        counter += 1
        data.append([t, T, G, pH, n_s])

    return np.array(data)

In [ ]:
def calculate_cost(Q, index, salt, vinegar, V_t, t_point, T_point, G_point, pH_point):
    standardData = simulationStandard(Q, index, salt, vinegar, V_t)
    standard_t = standardData[:,0]
    standard_T = standardData[:,1]
    standard_G = standardData[:,2]
    standard_pH = standardData[:,3]
    
    T_error = []
    G_error = []
    pH_error = []
    
    for index, t in enumerate(t_point):
        try:
            T_error.append((standard_T[int(t*10)] - T_point[index])**2)
            G_error.append((standard_G[int(t*10)]*1000 - G_point[index])**2)
            pH_error.append((standard_pH[int(t*10)] - pH_point[index])**2)
        except:
            pass

    print(np.average(T_error), np.average(G_error), np.average(pH_error))
    print(np.average(T_error)*0.01 + np.average(G_error) + np.average(pH_error)*20)

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(18, 4))
ax1.set_title("Temperature")
ax2.set_title("Conductance")
ax3.set_title("pH")

colors = ["Blue", "Orange", "Green", "Red", "Purple", "Brown"]

#input
Q = np.array([1]*3700 + [0.25]*7300) * 4.88*2**7
index = np.array([600, 750])
salt = np.array([0.2, 0]) * 10
vinegar = np.array([0, 0.3]) * 10

#calculate error
start = 7

t_point, T_point, G_point, pH_point = plot_utils.analyse_and_plot("Data/0518-4--pH--G--T.npy", False, 4.5, RTD_constant = 0.3)
t_point = t_point[start:]
T_point = T_point[start:]
G_point = G_point[start:]
pH_point = pH_point[start:]
calculate_cost(Q, index, salt, vinegar, 0.4, t_point, T_point, G_point, pH_point)
standardData = simulationStandard(Q, index, salt, vinegar, 0.4)

#plot
number = 0
ax1.plot(standardData[:,0], standardData[:,1], color = colors[number])
ax1.scatter(t_point[start:], T_point[start:], color = colors[number], marker = "x")
ax2.plot(standardData[:,0], standardData[:,2]*1000, color = colors[number])
ax2.scatter(t_point[start:], G_point[start:], color = colors[number], marker = "x")
ax3.plot(standardData[:,0], standardData[:,3], color = colors[number])
ax3.scatter(t_point[start:], pH_point[start:], color = colors[number], marker = "x")



#input
Q = np.array([1]*5000 + [0.25]*4000) * 4.88*2**7
index = np.array([300, 400, 500])
salt = np.array([1, 1, 1])
vinegar = np.array([1, 1, 1])

#calculate error
start = 5

t_point, T_point, G_point, pH_point = plot_utils.analyse_and_plot("Data/0529-2--pH--G--T.npy", False, 4.5, RTD_constant = 0.32)
t_point = t_point[start:]
T_point = T_point[start:]
G_point = G_point[start:]
pH_point = pH_point[start:]
calculate_cost(Q, index, salt, vinegar, 0.4, t_point, T_point, G_point, pH_point)
standardData = simulationStandard(Q, index, salt, vinegar, 0.4)

#plot
number = 1
ax1.plot(standardData[:,0], standardData[:,1], color = colors[number])
ax1.scatter(t_point[start:], T_point[start:], color = colors[number], marker = "x")
ax2.plot(standardData[:,0], standardData[:,2]*1000, color = colors[number])
ax2.scatter(t_point[start:], G_point[start:], color = colors[number], marker = "x")
ax3.plot(standardData[:,0], standardData[:,3], color = colors[number])
ax3.scatter(t_point[start:], pH_point[start:], color = colors[number], marker = "x")


#input
Q = np.array([1]*5000 + ([1]*150 + [0]*150) * 15) * 4.88*2**6
index = np.array([0])
salt = np.array([0])
vinegar = np.array([0])

#calculate error
start = 3

t_point, T_point, G_point, pH_point = plot_utils.analyse_and_plot("Data/0529-5--pH--G--T.npy", False, 4.05, RTD_constant = 0.3, T_backwards = -1, G_forwards = -1)
t_point = t_point[start:]
T_point = T_point[start:]
G_point = G_point[start:]
pH_point = pH_point[start:]
calculate_cost(Q, index, salt, vinegar, 0.134, t_point, T_point, G_point, pH_point)
standardData = simulationStandard(Q, index, salt, vinegar, 0.134)

#plot
number = 4
ax1.plot(standardData[:,0], standardData[:,1], color = colors[number])
ax1.scatter(t_point[start:], T_point[start:], color = colors[number], marker = "x")
ax2.plot(standardData[:,0], standardData[:,2]*1000, color = colors[number])
ax2.scatter(t_point[start:], G_point[start:], color = colors[number], marker = "x")
ax3.plot(standardData[:,0], standardData[:,3], color = colors[number])
ax3.scatter(t_point[start:], pH_point[start:], color = colors[number], marker = "x")